In [1]:
# a bunch of import statements
import numpy as np
import pandas as pd
import os
import sys
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from plotting import normalize_image, plot_image_array, plot_confusion_matrix, plot_model_history
import warnings
warnings.filterwarnings("ignore")

# configure GPU memory options (for Michael)
import tensorflow as tf
tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10240)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

2023-03-19 15:07:41.781515: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 15:07:42.868017: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-19 15:07:42.870817: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-19 15:07:42.870920: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [2]:
# set various paths
data_path = os.path.join(sys.path[0], 'galaxy-zoo-the-galaxy-challenge/data')
training_path = os.path.join(data_path, 'images_training_rev1')
test_path = os.path.join(data_path, 'images_test_rev1')

# find file names
training_file_names = os.listdir(training_path)
test_file_names = os.listdir(test_path)

In [3]:
# load csv data
all_ones_benchmark = pd.read_csv(os.path.join(data_path, 'all_ones_benchmark.csv'), dtype=str)
all_zeros_benchmark = pd.read_csv(os.path.join(data_path, 'all_zeros_benchmark.csv'), dtype=str)
central_pixel_benchmark = pd.read_csv(os.path.join(data_path, 'central_pixel_benchmark.csv'), dtype=str)
training_solutions_rev1 = pd.read_csv(os.path.join(data_path, 'training_solutions_rev1.csv'), dtype=str)

In [4]:
'''# For Evelyn
training_path = os.path.join(sys.path[0], 'images_training_rev1')
test_path = os.path.join(sys.path[0], 'images_test_rev1')
training_file_names = os.listdir(training_path)
test_file_names = os.listdir(test_path)


all_ones_benchmark = pd.read_csv(os.path.join(sys.path[0], 'all_ones_benchmark.csv'), dtype=str)
all_zeros_benchmark = pd.read_csv(os.path.join(sys.path[0], 'all_zeros_benchmark.csv'), dtype=str)
central_pixel_benchmark = pd.read_csv(os.path.join(sys.path[0], 'central_pixel_benchmark.csv'), dtype=str)
training_solutions_rev1 = pd.read_csv(os.path.join(sys.path[0], 'training_solutions_rev1.csv'), dtype=str)'''

"# For Evelyn\ntraining_path = os.path.join(sys.path[0], 'images_training_rev1')\ntest_path = os.path.join(sys.path[0], 'images_test_rev1')\ntraining_file_names = os.listdir(training_path)\ntest_file_names = os.listdir(test_path)\n\n\nall_ones_benchmark = pd.read_csv(os.path.join(sys.path[0], 'all_ones_benchmark.csv'), dtype=str)\nall_zeros_benchmark = pd.read_csv(os.path.join(sys.path[0], 'all_zeros_benchmark.csv'), dtype=str)\ncentral_pixel_benchmark = pd.read_csv(os.path.join(sys.path[0], 'central_pixel_benchmark.csv'), dtype=str)\ntraining_solutions_rev1 = pd.read_csv(os.path.join(sys.path[0], 'training_solutions_rev1.csv'), dtype=str)"

In [5]:
# load training images + sol
training_img = []
training_img_sol = []

for file in training_file_names[:10500]: 
    # extract training image
    training_img.append(cv2.imread(os.path.join(training_path, file)))
    
    # extract corresponding galaxy class probabilities from answer key
    galaxy_id = file[:6]
    sol_data = training_solutions_rev1.loc[training_solutions_rev1['GalaxyID'] == galaxy_id]
    training_img_sol.append(np.array(sol_data.iloc[0].tolist()[1:]))

# convert to numpy arrays
training_img = np.array(training_img)
training_img_sol = np.array(training_img_sol)

# load testing images
test_img = []
for file in test_file_names[:1000]:
    test_img.append(cv2.imread(os.path.join(test_path, file)))
test_img = np.array(test_img)

# class types (I'm sure there's a better way of doing this)
galaxy_class_labels = np.array(["Class1.1", "Class1.2", "Class1.3", "Class2.1", "Class2.2", "Class3.1", "Class3.2", 
                "Class4.1", "Class4.2", "Class5.1", "Class5.2", "Class5.3", "Class5.4", "Class6.1",
                "Class6.2", "Class7.1", "Class7.2", "Class7.3", "Class8.1", "Class8.2", "Class8.3",
                "Class8.4", "Class8.5", "Class8.6", "Class8.7", "Class9.1", "Class9.2", "Class9.3",
                "Class10.1", "Class10.2", "Class10.3", "Class11.1", "Class11.2", "Class11.3", "Class11.4",
                "Class11.5", "Class11.6"])

In [6]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


def reshape_arrays(data, labels):
    """reshape arrays for Keras"""
    data = data.reshape(-1, 424, 424, 3)
    labels = to_categorical(labels)
    return data, labels

# split the samples into training, validation and, test data sets:
train_frac = 0.8
val_frac = 0.2

# format the data for NN training
data_train_val, data_test, class_train_val, class_test = train_test_split(training_img, training_img_sol, test_size=None)

data_train, data_val, class_train, class_val = train_test_split(
    training_img, training_img_sol, test_size=val_frac / (train_frac + val_frac)
)

class_train = np.asarray(class_train, dtype=np.float64)

# print number of samples
print(
    f"Number of samples in the training ({data_train.shape[0]}); test ({data_train.shape[0]}); and validation ({data_val.shape[0]}) data sets"
)

Number of samples in the training (8400); test (8400); and validation (2100) data sets


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.utils import to_categorical

'''VGG6'''
def vgg6(input_shape=(424, 424, 3), n_classes: int = 37):
    """
        VGG6
    :param input_shape:
    :param n_classes:
    :return:
    """

    model = Sequential(name="VGG6")
    # input: 21x21 images with 1 channel -> (21, 21, 1) tensors.
    # this applies 16 convolution filters of size 3x3 each.
    model.add(Conv2D(16, (3, 3), activation="relu", input_shape=input_shape, name="conv1"))
    model.add(Conv2D(16, (3, 3), activation="relu", name="conv2"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3), activation="relu", name="conv3"))
    model.add(Conv2D(32, (3, 3), activation="relu", name="conv4"))
    model.add(BatchNormalization(axis=3, name="bn_2"))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(256, activation="relu", name="fc_1"))
    model.add(Dropout(0.5))
    # output layer
    activation = "sigmoid" if n_classes == 1 else "softmax"
    model.add(Dense(n_classes, activation=activation, name="fc_out"))

    return model

In [8]:
final_model = vgg6() # for now choose between vgg6() and ResNet50()
final_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
final_model.summary()

Model: "VGG6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 422, 422, 16)      448       
                                                                 
 conv2 (Conv2D)              (None, 420, 420, 16)      2320      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 210, 210, 16)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 210, 210, 16)      0         
                                                                 
 conv3 (Conv2D)              (None, 208, 208, 32)      4640      
                                                                 
 conv4 (Conv2D)              (None, 206, 206, 32)      9248      
                                                              

2023-03-19 15:08:19.044836: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 15:08:19.045646: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-19 15:08:19.045787: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-19 15:08:19.045881: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [9]:
# Train model for 20 epochs
n_epochs = 20
batch_size = 64

final_model_history = final_model.fit(
    data_train,
    class_train,
    epochs=n_epochs,
    batch_size=batch_size,
    verbose=1,
    validation_data=None,
    shuffle=True,
)
classes = final_model.predict(data_test)

2023-03-19 15:08:19.348144: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 4530355200 exceeds 10% of free system memory.
2023-03-19 15:08:21.944925: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 4530355200 exceeds 10% of free system memory.


Epoch 1/20


2023-03-19 15:08:23.470471: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inVGG6/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-03-19 15:08:23.922518: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-19 15:08:24.364273: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.85GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-03-19 15:08:24.364290: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.85GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-03-19 15:08

132/132 [==============================] - 20s 126ms/step - loss: 0.9160 - accuracy: 0.2771
Epoch 2/20
132/132 [==============================] - 16s 120ms/step - loss: 0.4802 - accuracy: 0.5481
Epoch 3/20
132/132 [==============================] - 16s 120ms/step - loss: 0.4234 - accuracy: 0.5755
Epoch 4/20
132/132 [==============================] - 16s 120ms/step - loss: 0.3847 - accuracy: 0.6013
Epoch 5/20
132/132 [==============================] - 16s 120ms/step - loss: 0.3733 - accuracy: 0.6012
Epoch 6/20
132/132 [==============================] - 16s 122ms/step - loss: 0.3683 - accuracy: 0.6014
Epoch 7/20
132/132 [==============================] - 16s 121ms/step - loss: 0.3652 - accuracy: 0.6014
Epoch 8/20
132/132 [==============================] - 16s 121ms/step - loss: 0.3562 - accuracy: 0.6015
Epoch 9/20
132/132 [==============================] - 16s 120ms/step - loss: 0.3520 - accuracy: 0.6017
Epoch 10/20
132/132 [==============================] - 16s 120ms/step - loss: 0.3499

2023-03-19 15:13:44.777299: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1415736000 exceeds 10% of free system memory.
2023-03-19 15:13:45.611045: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1415736000 exceeds 10% of free system memory.


83/83 [==============================] - 1s 11ms/step


In [10]:
class_train[0,:], class_train[26,:], class_train[135,:], class_train[6,:], class_train.shape

(array([0.010747  , 0.982675  , 0.006578  , 0.982675  , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.086518  , 0.913482  ,
        0.        , 0.        , 0.010747  , 0.        , 0.        ,
        0.        , 0.043259  , 0.043259  , 0.        , 0.        ,
        0.68686919, 0.26733084, 0.02847497, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]),
 array([0.005921  , 0.994079  , 0.        , 0.02775866, 0.96632034,
        0.85074843, 0.11557191, 0.86417448, 0.10214586, 0.01704106,
        0.37482889, 0.57445038, 0.        , 0.167988  , 0.832012  ,
        0.        , 0.005921  , 0.        , 0.09675907, 0.        ,
        0.02645643, 0.02573425, 0.01903825, 0.        , 0.        ,
        0.02775866, 0.        , 0.        , 0.20598204, 0.5408652 ,
        0.11732724, 0.04033534, 0.74710822, 0.        , 0.        ,
        0.    